# **In this task, we have a new goal!**

Build an easy and simple model that allows the decision-makers to predict the number of expected customer views in the next two months, determine the expected peak period, and identify the potential customers.




# STC Jawwy

In [ ]:
"""
Here we install libraries that are not installed by default
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb
!pip install prophet
!pip install plotly


# 1. Setup and Data Loading

In [ ]:
# Import the required libraries
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Jawwy dataset
The dataset includes total watching hours for customers per day.

You are required to work on predecting the forecast for the watching hours.

In [ ]:
dataframe = pd.read_excel("stc TV Data Set_T2 .xlsx",index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use
df=dataframe.copy()

# 2. Data Exploration

In [ ]:
# check the data shape
df.shape

In [ ]:
# display the first 5 rows
df.head()

In [ ]:
# display the dataset after applying data types
df.head()

In [ ]:
# describe the numeric values in the dataset
df.describe()

In [ ]:
# check if any column has null value in the dataset
df.isnull().any()

In [ ]:
# we import Visualization libraries
# you can ignore and use any other graphing libraries
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Setting the date as index
df.set_index('date_', inplace=True)

In [ ]:
# Display the dataframe after setting the date as index
df.head()

In [ ]:
# show the dataframe
fig = px.line(df,  y="Total_watch_time_in_houres")
fig.show()

In [ ]:
"""
TODO using the previous dataset (df) build a prediction model to predict the expected watch time for the next two months
Hint: you can build a forecast model to predict the results
"""

# 3. Time Series Forecasting with Prophet

In [ ]:
"""
Build and train the Prophet model, make predictions
"""
model = Prophet(weekly_seasonality=True, yearly_seasonality=True)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(df)

future = model.make_future_dataframe(periods=60)
forecast = model.predict(future)

# 4. Model Evaluation

In [ ]:
mae = mean_absolute_error(df['y'], forecast['yhat'][:len(df)])
rmse = np.sqrt(mean_squared_error(df['y'], forecast['yhat'][:len(df)]))
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

df_cv = cross_validation(model, initial='30 days', period='7 days', horizon = '30 days')
df_p = performance_metrics(df_cv)
print(df_p.head())

# 5. Forecast Visualization and Analysis

In [ ]:
"""
Visualize the forecast and analyze trends
"""
# Plot forecast
fig = model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), model, forecast)
plt.show()

In [ ]:
# Plot components
fig = model.plot_components(forecast)
plt.show()

In [ ]:
# Analyze next two months
next_two_months = forecast[forecast['ds'] > df['ds'].max()].reset_index(drop=True)
peak_periods = next_two_months.sort_values(by='yhat', ascending=False).head(5)

print("Top 5 Peak Viewing Days in Next Two Months:")
print(peak_periods[['ds', 'yhat']].to_string(index=False))


In [ ]:
# Weekly patterns
weekday_avg = next_two_months.groupby(next_two_months['ds'].dt.dayofweek)['yhat'].mean()
best_weekday = weekday_avg.idxmax()
worst_weekday = weekday_avg.idxmin()

print(f"\nBest day for viewership: {['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][best_weekday]}")
print(f"Worst day for viewership: {['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][worst_weekday]}")


# 6. Additional Insights

In [ ]:
"""
Generate heatmap and calculate growth rate
"""
# Heatmap of weekday vs. hour
df['weekday'] = df['ds'].dt.weekday
df['hour'] = df['ds'].dt.hour
pivot = df.pivot_table(values='y', index='weekday', columns='hour', aggfunc='mean')
plt.figure(figsize=(12,8))
sns.heatmap(pivot, cmap='YlOrRd')
plt.title('Average Watch Time by Weekday and Hour')
plt.show()

In [ ]:
# Calculate growth rate
growth_rate = (next_two_months['yhat'].iloc[-1] - next_two_months['yhat'].iloc[0]) / next_two_months['yhat'].iloc[0] * 100
print(f"\nPredicted growth rate over next two months: {growth_rate:.2f}%")


# 7. Actionable Recommendations

In [ ]:
"""
Provide actionable insights based on the analysis
"""
peak_day = weekday_avg.idxmax()
peak_day_name = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][peak_day]
print(f"Recommendation: Schedule new episode releases on {peak_day_name} to maximize viewership.")


# **Forecasted Watch Time Analysis for the Next Two Months**

**Viewership Trend:**

The forecast model predicts a significant increase in watch time over the next two months, with a projected growth rate of 446.94%. This substantial growth suggests a rapidly expanding user base or increased engagement from existing users.

**Peak Viewing Periods:**

The top 5 peak viewing days are all in late June, with June 22nd showing the highest predicted watch time of 7,036.93 hours.
Friday consistently emerges as the best day for viewership, while Saturday shows the lowest average watch time.


**Weekly Patterns:**

Weekdays, particularly towards the end of the week, show higher viewership compared to weekends.
There's a notable drop in viewership on Saturdays, indicating an opportunity for targeted content strategies.


**Actionable Insights:**

a) Content Scheduling:

Schedule new episode releases and high-priority content on Fridays to maximize viewership.
Consider special programming or promotions for Saturdays to boost traditionally low viewership.

b) User Engagement:

Develop strategies to capitalize on the projected growth, such as personalized recommendations or interactive features to retain new users.
Create targeted campaigns for weekday evenings when viewership peaks.

c) Content Strategy:

Analyze the types of content driving the substantial growth and invest in similar programming.
Develop weekend-specific content to address the viewership dip, particularly on Saturdays.

d) Marketing and Promotions:

Focus marketing efforts on the days leading up to peak viewing periods, especially for the identified top 5 days in June.
Consider loyalty programs or special offers to encourage consistent viewership throughout the week.


**Thank you for reading, Rinad Hassan**

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf


In [ ]:
colab_pdf('RinadHassan_stc TV_T2.ipynb')